# Linear Cryptanalysis and Differential Cryptanalysis

This is based on the paper found in: https://eprint.iacr.org/2017/1008.pdf

In [6]:
import string
import math
import random

## S-Boxes , P-Boxes and SPN

In [7]:
Sbox_dict = {'0000': '1110', 
        '0001': '0100', 
        '0010': '1101', 
        '0011': '0001', 
        '0100': '0010', 
        '0101': '1111', 
        '0110': '1011', 
        '0111': '1000',
        '1000': '0011', 
        '1001': '1010',
        '1010': '0110',
        '1011': '1100',
        '1100': '0101',
        '1101': '1001',
        '1110': '0000',
        '1111': '0111'}

inverse_Sbox_dict = {'1110': '0000', 
        '0100': '0001', 
        '1101': '0010', 
        '0001': '0011', 
        '0010': '0100', 
        '1111': '0101', 
        '1011': '0110', 
        '1000': '0111',
        '0011': '1000', 
        '1010': '1001',
        '0110': '1010',
        '1100': '1011',
        '0101': '1100',
        '1001': '1101',
        '0000': '1110',
        '0111': '1111'}

Pbox_dict = (1,5,9,13,2,6,10,14,3,7,11,15,4,8,12,16) #index:original position, element:permutated position

In [8]:
def Sbox(text): 
    assert len(text) == 4 
    if text in Sbox_dict: 
        return Sbox_dict[text]

def inverse_Sbox(text): 
    assert len(text) == 4 
    if text in inverse_Sbox_dict: 
        return inverse_Sbox_dict[text]

def Pbox(text): 
    assert len(text) == 16
    new_list = ['']*16
    for i in range(len(text)):
        perm_index = Pbox_dict[i]
        new_list[perm_index-1] = text[i]
    text= ''
    for j in new_list: 
        text+= j
    return text

def AddKey(text,key): ### Xor of length 16
    assert len(text) == 16 & len(key)==16
    x = int(text,2)
    x =  "{0:016b}".format(x^int(key,2))
    return x

def Xor4(text1,text2):  ### Xor of length 4
    assert len(text1) == 4 & len(text2) == 4
    x = int(text1,2)
    x =  "{0:04b}".format(x^int(text2,2))
    return x

In [86]:
def SPN(plaintext):
    assert len(plaintext) == 16
    text = plaintext
    rounds = 1
    k1='0100'*4
    k2='00100100'*2
    k3='00000100'*2
    while (rounds !=4):
        a = Sbox(text[0:4])
        b = Sbox(text[4:8])
        c = Sbox(text[8:12])
        d = Sbox(text[12:])
        text = a+b+c+d
        text = Pbox(text)
       
        ##add round key 
        if rounds ==1:
            text = AddKey(text, k1)
        if rounds ==2:
            text = AddKey(text, k2)
        if rounds ==3:
            text = AddKey(text, k3)   

        rounds+=1
        
    a4 = Sbox(text[0:4])
    b4 = Sbox(text[4:8])
    c4 = Sbox(text[8:12])
    d4 = Sbox(text[12:16])
    text = a4+b4+c4+d4
    ##add round key
    text = AddKey(text,'0100001100000111') #### This is the key we trying to break [15 6]
    return text

## Linear Cryptanalysis

We will only jump the analysis part of making the tables and tracing based on the tutorial analysis

In [10]:
def rand_text(length):  
    key1 = "" 
    for i in range(length): 
        temp = str(random.randint(0, 1)) # randint function to generate 0, 1 randomly and converting  
        key1 += temp 
    return key1

In [87]:
def bias_linear(key): 
    total = 0
    count = 0 
    partial_key1 = key[0:4]
    partial_key2 = key[4:8]
    partial_key3 = key[8:12]
    partial_key4 = key[12:16]
    while(total <10000):
        plaintext = rand_text(16) ### Linear differential is a known-plaintext attack, randomly find 10000 plaintext
        ciphertext = SPN(plaintext) 
        ##this part is based on the manual calculation
        partial_ctext2 = ciphertext[4:8]
        partial_ctext4 = ciphertext[12:16]
        partial_U2 = inverse_Sbox(Xor4(partial_ctext2, partial_key2))
        partial_U4 = inverse_Sbox(Xor4(partial_ctext4, partial_key4))
        p5,p7,p8 = plaintext[4],plaintext[6], plaintext[7]
        u6,u8,u14,u16 = partial_U2[1],partial_U2[3], partial_U4[1], partial_U4[3]  
        if int(u6)^int(u8)^int(u14)^int(u16)^int(p5)^int(p7)^int(p8) == 0:
            count+=1
        total+=1
    return abs(count - 5000)/10000

In [88]:
table = []
for i in range(16): 
    string1 = "{0:04b}".format(i)
    row = []
    for j in range(16):
        string2 = "{0:04b}".format(j)
        plaintext = '0000'+string1 + '0000' + string2
        row.append(bias_linear(plaintext))
        print('done ' +str(i) +" "+ str(j))
    table.append(row)
print(table)

done 0 0
done 0 1
done 0 2
done 0 3
done 0 4
done 0 5
done 0 6
done 0 7
done 0 8
done 0 9
done 0 10
done 0 11
done 0 12
done 0 13
done 0 14
done 0 15
done 1 0
done 1 1
done 1 2
done 1 3
done 1 4
done 1 5
done 1 6
done 1 7
done 1 8
done 1 9
done 1 10
done 1 11
done 1 12
done 1 13
done 1 14
done 1 15
done 2 0
done 2 1
done 2 2
done 2 3
done 2 4
done 2 5
done 2 6
done 2 7
done 2 8
done 2 9
done 2 10
done 2 11
done 2 12
done 2 13
done 2 14
done 2 15
done 3 0
done 3 1
done 3 2
done 3 3
done 3 4
done 3 5
done 3 6
done 3 7
done 3 8
done 3 9
done 3 10
done 3 11
done 3 12
done 3 13
done 3 14
done 3 15
done 4 0
done 4 1
done 4 2
done 4 3
done 4 4
done 4 5
done 4 6
done 4 7
done 4 8
done 4 9
done 4 10
done 4 11
done 4 12
done 4 13
done 4 14
done 4 15
done 5 0
done 5 1
done 5 2
done 5 3
done 5 4
done 5 5
done 5 6
done 5 7
done 5 8
done 5 9
done 5 10
done 5 11
done 5 12
done 5 13
done 5 14
done 5 15
done 6 0
done 6 1
done 6 2
done 6 3
done 6 4
done 6 5
done 6 6
done 6 7
done 6 8
done 6 9
done 6 10


In [89]:
max_biaslin = 0
max_i = -1
max_j = -1
for i in range(16):
     for j in range(16):
        if table[i][j] >= max_biaslin: 
            max_biaslin = table[i][j]
            max_i = i
            max_j = j
            print(str(max_biaslin) +" "+ str(max_i)+ " " + str(max_j))

0.008 0 0
0.0211 0 7
0.0238 1 9
0.0325 3 7


Find the nearest to the bias 1/32 in magnitude based on the 

In [90]:
nearest_biaslin = 0
nearest_i = -1
nearest_j = -1
for i in range(16):
     for j in range(16):
        if abs(1/32 - table[i][j])<= abs(1/32 - nearest_biaslin): 
            nearest_biaslin = table[i][j]
            nearest_i = i
            nearest_j = j
            print(str(nearest_biaslin) +" "+ str(nearest_i)+ " " + str(nearest_j))

0.008 0 0
0.0211 0 7
0.0238 1 9
0.0325 3 7


This confirm that the highest partial key is the largest bias as the paper has written. Tried partial keys for [2,4] and [1,2].

## Differential Cryptanalysis

In [82]:
def prob_differential(target_key):
    assert len(target_key) ==16
    total = 0
    count = 0 
    partial_key1 = target_key[0:4]
    partial_key2 = target_key[4:8]
    partial_key3 = target_key[8:12]
    partial_key4 = target_key[12:16]
    while(total <5000) :
        differential = '0000101100000000' ## this is the differential we want. since differential cryptanalysis is a chosen plaintext attack
        plaintext1 = rand_text(16) ## we randomly find a plaintext.
        plaintext2 = "{0:016b}".format(int(plaintext1,2)^int(differential,2)) ## and get the differential pair of plaintext1.
        ciphertext1 = SPN(plaintext1)
        ciphertext2 = SPN(plaintext2)
        ##this part is based on the manual calculation in the paper
        differential_ciphertext = "{0:016b}".format(int(ciphertext1,2)^int(ciphertext2,2))
        if (differential_ciphertext[0:4] != '0000' or differential_ciphertext[8:12] != '0000'):
            total+=1
            continue
        diff_u2 = Xor4(inverse_Sbox(Xor4(ciphertext1[4:8],target_key[4:8])),inverse_Sbox(Xor4(ciphertext2[4:8],target_key[4:8])))
        diff_u4 = Xor4(inverse_Sbox(Xor4(ciphertext1[12:16],target_key[12:16])),inverse_Sbox(Xor4(ciphertext2[12:16],target_key[12:16])))
        if (diff_u2 == '0110' and diff_u4 == '0110'):
            count+=1
        total+=1
    return count/5000
        

In [83]:
print(prob_differential('0000001000000100'))

0.0


In [84]:
table_diff = []
for i in range(16):
    string_diff1 = "{0:04b}".format(i)
    row_diff = []
    for j in range(16):
        string_diff2 = "{0:04b}".format(j)
        plaintext_diff = '0000'+string_diff1 + '0000' + string_diff2
        row_diff.append(prob_differential(plaintext_diff))
        print('done ' +str(i) +" "+ str(j))
    table_diff.append(row_diff)
print(table)

done 0 0
done 0 1
done 0 2
done 0 3
done 0 4
done 0 5
done 0 6
done 0 7
done 0 8
done 0 9
done 0 10
done 0 11
done 0 12
done 0 13
done 0 14
done 0 15
done 1 0
done 1 1
done 1 2
done 1 3
done 1 4
done 1 5
done 1 6
done 1 7
done 1 8
done 1 9
done 1 10
done 1 11
done 1 12
done 1 13
done 1 14
done 1 15
done 2 0
done 2 1
done 2 2
done 2 3
done 2 4
done 2 5
done 2 6
done 2 7
done 2 8
done 2 9
done 2 10
done 2 11
done 2 12
done 2 13
done 2 14
done 2 15
done 3 0
done 3 1
done 3 2
done 3 3
done 3 4
done 3 5
done 3 6
done 3 7
done 3 8
done 3 9
done 3 10
done 3 11
done 3 12
done 3 13
done 3 14
done 3 15
done 4 0
done 4 1
done 4 2
done 4 3
done 4 4
done 4 5
done 4 6
done 4 7
done 4 8
done 4 9
done 4 10
done 4 11
done 4 12
done 4 13
done 4 14
done 4 15
done 5 0
done 5 1
done 5 2
done 5 3
done 5 4
done 5 5
done 5 6
done 5 7
done 5 8
done 5 9
done 5 10
done 5 11
done 5 12
done 5 13
done 5 14
done 5 15
done 6 0
done 6 1
done 6 2
done 6 3
done 6 4
done 6 5
done 6 6
done 6 7
done 6 8
done 6 9
done 6 10


In [85]:
max_probdiff = 0
max_i_diff = -1
max_j_diff = -1
for i in range(16):
     for j in range(16):
        if table_diff[i][j] >= max_probdiff: 
            max_probdiff = table_diff[i][j]
            max_i_diff = i
            max_j_diff = j
            print(str(max_probdiff) +" "+ str(max_i_diff)+ " " + str(max_j_diff))

0.0 0 0
0.0 0 1
0.0 0 2
0.0002 0 3
0.0002 0 4
0.0002 0 6
0.0124 1 0
0.0292 4 0


This confirm the paper that the largest prob is the key. Tried partial keys for [2,4] and [1,2]

## Results


1. '0000001000000100': expected output: [2 4]  Linear:0.0371 [2 4]                 Differential: 0.0276 [2 4]<br>
2. '0000000100000010': expected output: [1 2]  Linear:0.0324 [1 2]                 Differential:0.024 [1 2] <br>
3. '0100001100000111': expected output: [3 7]  Linear: 0.0261 [3 7] 0.0274 [3 10]  Differential: 0.0272 [3 7] <br>
4. '0100010101001111': expected output: [5 15] Linear: 0.0313 [5 15]               Differential: 0.0238 [5 15] <br>
5. '0100001100001010': expected output: [3 10] Linear: 0.0317 [3 10]               Differential: 0.0258 [3 10]  <br>
6. '0101111100000110': expected output: [15 6] Linear: 0.0352 [15 6]               Differential:  0.0274 [15 6]<br>
7. '1001101100001000': expected output: [11 8] Linear: 0.028 [11 8]                Differential: 0.0274 [11 8]<br>
8. '1011100111111001': expected output: [9 9]  Linear: 0.0277 [9 9] 0.032 [11 9]   Differential: 0.0236 [9 9] <br>
9. '1011101110001101': expected output: [11 13]Linear: 0.0321 [11 13]              Differential: 0.0282 [11 13] <br>
10. '1011010010000000': expected output: [4 0] Linear: 0.0275 [4 0]                Differential: 0.0292 [4 0]